In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import numpy.random as npr
import spglib as spg
import ase
from ase.io import read,write
from ase.spacegroup import crystal
from ase.visualize import view
import cPickle as pck
import pandas as pd
from tqdm import tqdm_notebook

from quippy.potential import Potential, Minim
from ase.optimize import  FIRE
from ase.constraints import UnitCellFilter

In [4]:
from generate_and_relax_structures import generate_crystal_step_1,generate_crystal_step_1_wrapper
from libs.utils import unskewCell,ase2qp,qp2ase,get_standard_frame
from libs.input_structure import input2crystal,getCellParam
from libs.LJ_pressure import make_LJ_input,LJ_vcrelax
from libs.raw_data import z2symb,z2VdWradius,z2Covalentradius,SG2BravaisLattice,WyckTable

In [5]:
import sys
sys.path.insert(0,'/local/git/glosim2/')
from libmatch.soap import get_Soaps
from libmatch.utils import get_soapSize,get_spkit,get_spkitMax,ase2qp,qp2ase
from libmatch.chemical_kernel import deltaKernel,PartialKernels
from GlobalSimilarity import get_environmentalKernels,get_globalKernel

In [6]:
d2r = np.pi/180.

In [7]:
def compare(frame1,frame2):
    centerweight  = 1.
    gaussian_width = 0.1
    cutoff = 3.5
    cutoff_transition_width = 0.5
    nmax = 8
    lmax = 6
    nocenters = []
    is_fast_average = False

    soap_params = {
              'centerweight': centerweight, 
              'gaussian_width': gaussian_width,'cutoff': cutoff, 
              'cutoff_transition_width': cutoff_transition_width,
              'nmax': nmax, 'lmax': lmax, 'is_fast_average':is_fast_average,
              'chem_channels': True ,'nocenters': nocenters,'dispbar':True,
                   }
    ff = []
    for frame in [frame1,frame2]:
        ff.append(ase2qp(frame))
     
    envk = get_environmentalKernels(ff,nthreads=1, nprocess=1, nchunks=1,**soap_params)
    gkern = get_globalKernel(envk, kernel_type='average', zeta=4, gamma=1.0, eps=1e-06, 
                             nthreads=8, normalize_global_kernel=True)
    
    return gkern[0,1]

# info

In [134]:
fileNames = {}
infoPath = './reference_info/'
structurePath = './structures/'
fileNames['crystals'] = structurePath + 'input_crystals_sg1-230-18-10-17.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [135]:
# with open(fileNames['crystals'],'rb') as f:
#     crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

# make some info dict

In [58]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [59]:
ElemTable.head()

,covr,elneg,mass,sym,val,vdwr,z
0,0.32,2.20,1.007940,H,1,1.20,1
1,0.46,0.00,4.002602,He,0,1.43,2
2,1.33,0.98,6.941000,Li,1,2.12,3
3,1.02,1.57,9.012182,Be,2,1.98,4
4,0.85,2.04,10.811000,B,3,1.91,5


In [61]:
z2eleminfo = {z:{'symbol':symb,'mass':mass,'valence':val} for z,symb,mass,val in ElemTable[['z','sym','mass','val']].values}
pp.pprint(z2eleminfo)

{   1: {   'mass': 1.00794, 'symbol': 'H', 'valence': 1},
    2: {   'mass': 4.002602, 'symbol': 'He', 'valence': 0},
    3: {   'mass': 6.941, 'symbol': 'Li', 'valence': 1},
    4: {   'mass': 9.012182, 'symbol': 'Be', 'valence': 2},
    5: {   'mass': 10.811, 'symbol': 'B', 'valence': 3},
    6: {   'mass': 12.0107, 'symbol': 'C', 'valence': 4},
    7: {   'mass': 14.0067, 'symbol': 'N', 'valence': 3},
    8: {   'mass': 15.9994, 'symbol': 'O', 'valence': 2},
    9: {   'mass': 18.9984032, 'symbol': 'F', 'valence': 1},
    10: {   'mass': 20.1797, 'symbol': 'Ne', 'valence': 0},
    11: {   'mass': 22.98977, 'symbol': 'Na', 'valence': 1},
    12: {   'mass': 24.305, 'symbol': 'Mg', 'valence': 2},
    13: {   'mass': 26.981538, 'symbol': 'Al', 'valence': 3},
    14: {   'mass': 28.0855, 'symbol': 'Si', 'valence': 4},
    15: {   'mass': 30.973761, 'symbol': 'P', 'valence': 5},
    16: {   'mass': 32.065, 'symbol': 'S', 'valence': 6},
    17: {   'mass': 35.453, 'symbol': 'Cl', 'valence

In [42]:
z2symb = {z:symb for z,symb in ElemTable[['z','sym']].values}
print z2symb

{1: 'H', 2: 'He', 3: 'Li', 4: 'Be', 5: 'B', 6: 'C', 7: 'N', 8: 'O', 9: 'F', 10: 'Ne', 11: 'Na', 12: 'Mg', 13: 'Al', 14: 'Si', 15: 'P', 16: 'S', 17: 'Cl', 18: 'Ar', 19: 'K', 20: 'Ca', 21: 'Sc', 22: 'Ti', 23: 'V', 24: 'Cr', 25: 'Mn', 26: 'Fe', 27: 'Co', 28: 'Ni', 29: 'Cu', 30: 'Zn', 31: 'Ga', 32: 'Ge', 33: 'As', 34: 'Se', 35: 'Br', 36: 'Kr', 37: 'Rb', 38: 'Sr', 39: 'Y', 40: 'Zr', 41: 'Nb', 42: 'Mo', 43: 'Tc', 44: 'Ru', 45: 'Rh', 46: 'Pd', 47: 'Ag', 48: 'Cd', 49: 'In', 50: 'Sn', 51: 'Sb', 52: 'Te', 53: 'I', 54: 'Xe', 55: 'Cs', 56: 'Ba', 57: 'La', 58: 'Ce', 59: 'Pr', 60: 'Nd', 62: 'Sm', 63: 'Eu', 64: 'Gd', 65: 'Tb', 66: 'Dy', 67: 'Ho', 68: 'Er', 69: 'Tm', 70: 'Yb', 71: 'Lu', 72: 'Hf', 73: 'Ta', 74: 'W', 75: 'Re', 76: 'Os', 77: 'Ir', 78: 'Pt', 79: 'Au', 80: 'Hg', 81: 'Tl', 82: 'Pb', 83: 'Bi'}


In [70]:
from mendeleev import element

In [71]:
el = element(14)
print el.vdw_radius*100

21000.0


In [63]:
z2VdWradius = {}
for z in ElemTable[['z']].values:
    try:
        z2VdWradius[z[0]] = element(z[0]).vdw_radius/100.
    except:
        print z[0]

In [64]:
print z2VdWradius

{1: 1.1, 2: 1.4, 3: 1.82, 4: 1.53, 5: 1.92, 6: 1.7, 7: 1.55, 8: 1.52, 9: 1.47, 10: 1.54, 11: 2.27, 12: 1.73, 13: 1.84, 14: 2.1, 15: 1.8, 16: 1.8, 17: 1.75, 18: 1.88, 19: 2.75, 20: 2.31, 21: 2.15, 22: 2.11, 23: 2.07, 24: 2.06, 25: 2.05, 26: 2.04, 27: 2.0, 28: 1.97, 29: 1.96, 30: 2.01, 31: 1.87, 32: 2.11, 33: 1.85, 34: 1.9, 35: 1.85, 36: 2.02, 37: 3.03, 38: 2.49, 39: 2.32, 40: 2.23, 41: 2.18, 42: 2.17, 43: 2.16, 44: 2.13, 45: 2.1, 46: 2.1, 47: 2.11, 48: 2.18, 49: 1.93, 50: 2.17, 51: 2.06, 52: 2.06, 53: 1.98, 54: 2.16, 55: 3.43, 56: 2.68, 57: 2.43, 58: 2.42, 59: 2.4, 60: 2.39, 62: 2.36, 63: 2.35, 64: 2.34, 65: 2.33, 66: 2.31, 67: 2.3, 68: 2.29, 69: 2.27, 70: 2.26, 71: 2.24, 72: 2.23, 73: 2.22, 74: 2.18, 75: 2.16, 76: 2.16, 77: 2.13, 78: 2.13, 79: 2.14, 80: 2.23, 81: 1.96, 82: 2.02, 83: 2.07}


In [65]:
z2Covalentradius = {}
for z in ElemTable[['z']].values:
    try:
        z2Covalentradius[z[0]] = element(z[0]).covalent_radius_pyykko/100.
    except:
        print z[0]

In [66]:
print z2Covalentradius

{1: 0.32, 2: 0.46, 3: 1.33, 4: 1.02, 5: 0.85, 6: 0.75, 7: 0.71, 8: 0.63, 9: 0.64, 10: 0.67, 11: 1.55, 12: 1.39, 13: 1.26, 14: 1.16, 15: 1.11, 16: 1.03, 17: 0.99, 18: 0.96, 19: 1.96, 20: 1.71, 21: 1.48, 22: 1.36, 23: 1.34, 24: 1.22, 25: 1.19, 26: 1.16, 27: 1.11, 28: 1.1, 29: 1.12, 30: 1.18, 31: 1.24, 32: 1.21, 33: 1.21, 34: 1.16, 35: 1.14, 36: 1.17, 37: 2.1, 38: 1.85, 39: 1.63, 40: 1.54, 41: 1.47, 42: 1.38, 43: 1.28, 44: 1.25, 45: 1.25, 46: 1.2, 47: 1.28, 48: 1.36, 49: 1.42, 50: 1.4, 51: 1.4, 52: 1.36, 53: 1.33, 54: 1.31, 55: 2.32, 56: 1.96, 57: 1.8, 58: 1.63, 59: 1.76, 60: 1.74, 62: 1.72, 63: 1.68, 64: 1.69, 65: 1.68, 66: 1.67, 67: 1.66, 68: 1.65, 69: 1.64, 70: 1.7, 71: 1.62, 72: 1.52, 73: 1.46, 74: 1.37, 75: 1.31, 76: 1.29, 77: 1.22, 78: 1.23, 79: 1.24, 80: 1.33, 81: 1.44, 82: 1.44, 83: 1.51}


In [59]:
z2FormationEnergy = {}
for z in ElemTable[['z']].values:
    try:
        z2FormationEnergy[z[0]] = element(z[0]).heat_of_formation
    except:
        print z[0]

In [60]:
print z2FormationEnergy

{1: 217.998, 2: None, 3: 159.3, 4: 324.0, 5: 565.0, 6: 716.87, 7: 472.44, 8: 249.229, 9: 79.335, 10: None, 11: 107.5, 12: 147.1, 13: 330.9, 14: 450.0, 15: 316.5, 16: 277.17, 17: 121.302, 18: None, 19: 89.0, 20: 177.8, 21: 377.8, 22: 473.0, 23: 515.5, 24: 397.48, 25: 283.3, 26: 415.5, 27: 426.7, 28: 430.1, 29: 337.4, 30: 130.4, 31: 271.96, 32: 372.0, 33: 302.5, 34: 227.2, 35: 111.85, 36: None, 37: 80.9, 38: 164.0, 39: 424.7, 40: 610.0, 41: 733.0, 42: 658.98, 43: 678.0, 44: 650.6, 45: 556.0, 46: 376.6, 47: 284.9, 48: 111.8, 49: 243.0, 50: 301.2, 51: 264.4, 52: 196.6, 53: 106.757, 54: None, 55: 76.5, 56: 179.1, 57: 431.0, 58: 420.1, 59: 356.9, 60: 326.9, 62: 206.7, 63: 177.4, 64: 397.5, 65: 388.7, 66: 290.4, 67: 300.6, 68: 316.4, 69: 232.2, 70: 155.6, 71: 427.6, 72: 618.4, 73: 782.0, 74: 851.0, 75: 774.0, 76: 787.0, 77: 669.0, 78: 565.7, 79: 368.2, 80: 61.38, 81: 182.2, 82: 195.2, 83: 209.6}


In [72]:
entry = WyckTable[120]
entry.to_dict().keys()

['Site symmetry', 'Multiplicity', 'Site generator', 'Wyckoff letter']

In [73]:
new_table = {}
for sg,entry in WyckTable.iteritems():
    new_table[sg] = entry.to_dict()

In [57]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [58]:
pp.pprint(new_table)

{   1: {   'Multiplicity': {   0: 1},
           'Site generator': {   0: ['x', 'y', 'z']},
           'Site symmetry': {   0: 1},
           'Wyckoff letter': {   0: 'a'}},
    2: {   'Multiplicity': {   0: 2,
                               1: 1,
                               2: 1,
                               3: 1,
                               4: 1,
                               5: 1,
                               6: 1,
                               7: 1,
                               8: 1},
           'Site generator': {   0: ['x', 'y', 'z'],
                                 1: [0.5, 0.5, 0.5],
                                 2: [0.0, 0.5, 0.5],
                                 3: [0.5, 0.0, 0.5],
                                 4: [0.5, 0.5, 0.0],
                                 5: [0.5, 0.0, 0.0],
                                 6: [0.0, 0.5, 0.0],
                                 7: [0.0, 0.0, 0.5],
                                 8: [0.0, 0.0, 0.0]},
           'Si

                                1: 4,
                                2: 4,
                                3: 4,
                                4: 4,
                                5: 4,
                                6: 4,
                                7: 2,
                                8: 2,
                                9: 2,
                                10: 2},
            'Site generator': {   0: ['x', 'y', 'z'],
                                  1: [0.0, 0.5, 'z'],
                                  2: [0.0, 0.0, 'z'],
                                  3: [0.5, 'y', 0.0],
                                  4: [0.0, 'y', 0.0],
                                  5: ['x', 0.0, 0.5],
                                  6: ['x', 0.0, 0.0],
                                  7: [0.0, 0.5, 0.0],
                                  8: [0.0, 0.0, 0.5],
                                  9: [0.5, 0.0, 0.0],
                                  10: [0.0, 0.0, 0.0]},
            'Site symm

    73: {   'Multiplicity': {   0: 16, 1: 8, 2: 8, 3: 8, 4: 8, 5: 8},
            'Site generator': {   0: ['x', 'y', 'z'],
                                  1: [0.0, 0.25, 'z'],
                                  2: [0.25, 'y', 0.0],
                                  3: ['x', 0.0, 0.25],
                                  4: [0.25, 0.25, 0.25],
                                  5: [0.0, 0.0, 0.0]},
            'Site symmetry': {   0: 1,
                                 1: '..2',
                                 2: '.2.',
                                 3: '2..',
                                 4: '-1',
                                 5: '-1'},
            'Wyckoff letter': {   0: 'f',
                                  1: 'e',
                                  2: 'd',
                                  3: 'c',
                                  4: 'b',
                                  5: 'a'}},
    74: {   'Multiplicity': {   0: 16,
                                1: 8,
               

                                   8: [0.0, 0.0, 0.25]},
             'Site symmetry': {   0: 1,
                                  1: '..2',
                                  2: '2..',
                                  3: '2..',
                                  4: '..2',
                                  5: '2.22',
                                  6: '-4..',
                                  7: '-4..',
                                  8: '2.22'},
             'Wyckoff letter': {   0: 'i',
                                   1: 'h',
                                   2: 'g',
                                   3: 'f',
                                   4: 'e',
                                   5: 'd',
                                   6: 'c',
                                   7: 'b',
                                   8: 'a'}},
    121: {   'Multiplicity': {   0: 16,
                                 1: 8,
                                 2: 8,
                                 3: 8,


                                 3: 6,
                                 4: 3,
                                 5: 3,
                                 6: 2,
                                 7: 2,
                                 8: 1,
                                 9: 1},
             'Site generator': {   0: ['x', 'y', 'z'],
                                   1: ['x', '-x', 'z'],
                                   2: ['x', 0.0, 0.5],
                                   3: ['x', 0.0, 0.0],
                                   4: [0.5, 0.0, 0.5],
                                   5: [0.5, 0.0, 0.0],
                                   6: [   0.3333333333333333,
                                          0.6666666666666666,
                                          'z'],
                                   7: [0.0, 0.0, 'z'],
                                   8: [0.0, 0.0, 0.5],
                                   9: [0.0, 0.0, 0.0]},
             'Site symmetry': {   0: 1,
                 

                                   3: ['x', 0.5, 0.0],
                                   4: ['x', 'x', 'x'],
                                   5: ['x', 0.5, 0.5],
                                   6: ['x', 0.0, 0.0],
                                   7: [0.5, 0.0, 0.0],
                                   8: [0.0, 0.5, 0.5],
                                   9: [0.5, 0.5, 0.5],
                                   10: [0.0, 0.0, 0.0]},
             'Site symmetry': {   0: 1,
                                  1: '..2',
                                  2: '..2',
                                  3: '2..',
                                  4: '.3.',
                                  5: '4..',
                                  6: '4..',
                                  7: '42.2',
                                  8: '42.2',
                                  9: '432',
                                  10: '432'},
             'Wyckoff letter': {   0: 'k',
                               

In [126]:
dont_print_wyck = [4, 7,9, 19, 19, 29, 33, 76, 78, 144,
                   145, 169, 170, 225, 226, 227, 228,]
aa = []
for sg in dont_print_wyck:
    table = new_table[sg]

    w = table['Wyckoff letter'][0]

    aa.append((sg,sgwyck2qewyck[(sg,w)]))
pp.pprint(aa)

[   (4, '2a'),
    (7, '2a'),
    (9, '4a'),
    (19, '4a'),
    (19, '4a'),
    (29, '4a'),
    (33, '4a'),
    (76, '4a'),
    (78, '4a'),
    (144, '3a'),
    (145, '3a'),
    (169, '6a'),
    (170, '6a'),
    (225, '192l'),
    (226, '192j'),
    (227, '192i'),
    (228, '192h')]


In [112]:
sgwyck2qewyck = {}
for sg,table in new_table.iteritems():
    #sgwyck2qewyck[sg] = {}
    for it in table['Multiplicity']:
        wyck,mult = table['Wyckoff letter'][it],table['Multiplicity'][it]
        sgwyck2qewyck[(sg,wyck)] = '{}{}'.format(mult,wyck)

In [77]:
pp.pprint(sgwyck2qewyck)

{   (1, 'a'): '1a',
    (2, 'a'): '1a',
    (2, 'b'): '1b',
    (2, 'c'): '1c',
    (2, 'd'): '1d',
    (2, 'e'): '1e',
    (2, 'f'): '1f',
    (2, 'g'): '1g',
    (2, 'h'): '1h',
    (2, 'i'): '2i',
    (3, 'a'): '1a',
    (3, 'b'): '1b',
    (3, 'c'): '1c',
    (3, 'd'): '1d',
    (3, 'e'): '2e',
    (4, 'a'): '2a',
    (5, 'a'): '2a',
    (5, 'b'): '2b',
    (5, 'c'): '4c',
    (6, 'a'): '1a',
    (6, 'b'): '1b',
    (6, 'c'): '2c',
    (7, 'a'): '2a',
    (8, 'a'): '2a',
    (8, 'b'): '4b',
    (9, 'a'): '4a',
    (10, 'a'): '1a',
    (10, 'b'): '1b',
    (10, 'c'): '1c',
    (10, 'd'): '1d',
    (10, 'e'): '1e',
    (10, 'f'): '1f',
    (10, 'g'): '1g',
    (10, 'h'): '1h',
    (10, 'i'): '2i',
    (10, 'j'): '2j',
    (10, 'k'): '2k',
    (10, 'l'): '2l',
    (10, 'm'): '2m',
    (10, 'n'): '2n',
    (10, 'o'): '4o',
    (11, 'a'): '2a',
    (11, 'b'): '2b',
    (11, 'c'): '2c',
    (11, 'd'): '2d',
    (11, 'e'): '2e',
    (11, 'f'): '4f',
    (12, 'a'): '2a',
    (12, 'b'): '2b

In [103]:
np.array(['x',0.5,1],dtype=np.dtype(object))

array(['x', 0.5, 1], dtype=object)

In [108]:
sgwyck2qemask = {}
for sg,table in new_table.iteritems():
    #sgwyck2qewyck[sg] = {}
    for it in table['Multiplicity']:
        wyck,mult,site_gen = table['Wyckoff letter'][it],table['Multiplicity'][it],WyckTable[sg]['Site generator'][it]
        sss = np.array(site_gen,dtype=np.dtype(object))
        mask = np.zeros((3,),bool)
#         if it>0:
#             if 'x' in sss[[1,2]]:
#                 print site_gen,sg
#             if 'y' in sss[[0,2]]:
#                 print site_gen,sg
#             if 'z' in sss[[1,0]]:
#                 print site_gen,sg
                
        for gen in site_gen:
            if isinstance(gen,float):
                continue
            if 'x' in gen:
                mask[0] = True
            if 'y' in gen:
                mask[1] = True
            if 'z' in gen:
                mask[2] = True
        sgwyck2qemask[(sg,'{}{}'.format(mult,wyck))] = list(mask)
pp.pprint(sgwyck2qemask)      

{   (1, '1a'): [True, True, True],
    (2, '1a'): [False, False, False],
    (2, '1b'): [False, False, False],
    (2, '1c'): [False, False, False],
    (2, '1d'): [False, False, False],
    (2, '1e'): [False, False, False],
    (2, '1f'): [False, False, False],
    (2, '1g'): [False, False, False],
    (2, '1h'): [False, False, False],
    (2, '2i'): [True, True, True],
    (3, '1a'): [False, True, False],
    (3, '1b'): [False, True, False],
    (3, '1c'): [False, True, False],
    (3, '1d'): [False, True, False],
    (3, '2e'): [True, True, True],
    (4, '2a'): [True, True, True],
    (5, '2a'): [False, True, False],
    (5, '2b'): [False, True, False],
    (5, '4c'): [True, True, True],
    (6, '1a'): [True, False, True],
    (6, '1b'): [True, False, True],
    (6, '2c'): [True, True, True],
    (7, '2a'): [True, True, True],
    (8, '2a'): [True, False, True],
    (8, '4b'): [True, True, True],
    (9, '4a'): [True, True, True],
    (10, '1a'): [False, False, False],
    (10, '1b

    (79, '2a'): [False, False, True],
    (79, '4b'): [False, False, True],
    (79, '8c'): [True, True, True],
    (80, '4a'): [False, False, True],
    (80, '8b'): [True, True, True],
    (81, '1a'): [False, False, False],
    (81, '1b'): [False, False, False],
    (81, '1c'): [False, False, False],
    (81, '1d'): [False, False, False],
    (81, '2e'): [False, False, True],
    (81, '2f'): [False, False, True],
    (81, '2g'): [False, False, True],
    (81, '4h'): [True, True, True],
    (82, '2a'): [False, False, False],
    (82, '2b'): [False, False, False],
    (82, '2c'): [False, False, False],
    (82, '2d'): [False, False, False],
    (82, '4e'): [False, False, True],
    (82, '4f'): [False, False, True],
    (82, '8g'): [True, True, True],
    (83, '1a'): [False, False, False],
    (83, '1b'): [False, False, False],
    (83, '1c'): [False, False, False],
    (83, '1d'): [False, False, False],
    (83, '2e'): [False, False, False],
    (83, '2f'): [False, False, False],
    (8

    (160, '9b'): [True, False, True],
    (161, '18b'): [True, True, True],
    (161, '6a'): [False, False, True],
    (162, '12l'): [True, True, True],
    (162, '1a'): [False, False, False],
    (162, '1b'): [False, False, False],
    (162, '2c'): [False, False, False],
    (162, '2d'): [False, False, False],
    (162, '2e'): [False, False, True],
    (162, '3f'): [False, False, False],
    (162, '3g'): [False, False, False],
    (162, '4h'): [False, False, True],
    (162, '6i'): [True, False, False],
    (162, '6j'): [True, False, False],
    (162, '6k'): [True, False, True],
    (163, '12i'): [True, True, True],
    (163, '2a'): [False, False, False],
    (163, '2b'): [False, False, False],
    (163, '2c'): [False, False, False],
    (163, '2d'): [False, False, False],
    (163, '4e'): [False, False, True],
    (163, '4f'): [False, False, True],
    (163, '6g'): [False, False, False],
    (163, '6h'): [True, False, False],
    (164, '12j'): [True, True, True],
    (164, '1a'): [Fa

In [109]:

dont_print_wyck_dict = {
    4: ['2a'],
    7: ['2a'],
    9: ['4a'],
    19: ['4a'],
    29: ['4a'],
    33: ['4a'],
    76: ['4a'],
    78: ['4a'],
    144: ['3a'],
    145: ['3a'],
    169: ['6a'],
    170: ['6a'],
    225: ['4a'],
    226: ['8a'],
    227: ['8a'],
    228: ['16a'],
}
aaa = []
for k,v in dont_print_wyck_dict.iteritems():
    aaa.append((k,v[0]))
pp.pprint(aaa)

[   (33, '4a'),
    (226, '8a'),
    (227, '8a'),
    (4, '2a'),
    (7, '2a'),
    (9, '4a'),
    (170, '6a'),
    (225, '4a'),
    (76, '4a'),
    (78, '4a'),
    (144, '3a'),
    (145, '3a'),
    (19, '4a'),
    (169, '6a'),
    (228, '16a'),
    (29, '4a')]


# make input

In [28]:
print WyckTable[120]
len(WyckTable[120])

   Multiplicity Wyckoff letter Site symmetry    Site generator
0            16              i             1         [x, y, z]
1             8              h           ..2   [x, x+0.5, 0.0]
2             8              g           2..     [0.0, 0.5, z]
3             8              f           2..     [0.0, 0.0, z]
4             8              e           ..2      [x, x, 0.25]
5             4              d          2.22   [0.0, 0.5, 0.0]
6             4              c          -4..  [0.0, 0.5, 0.25]
7             4              b          -4..   [0.0, 0.0, 0.0]
8             4              a          2.22  [0.0, 0.0, 0.25]


9

## make the initial srtucture

In [99]:
cc = crystal(symbols=['Si'], basis=[0.1,0.4,0.6], spacegroup=86, cellpar=[1,1,1,90,90,90],symprec=1e-5, pbc=True)
print cc.get_scaled_positions()

[[ 0.1  0.4  0.6]
 [ 0.9  0.6  0.6]
 [ 0.1  0.6  0.1]
 [ 0.9  0.4  0.1]
 [ 0.4  0.1  0.9]
 [ 0.6  0.9  0.9]
 [ 0.4  0.9  0.4]
 [ 0.6  0.1  0.4]]


In [97]:
seed = 53485
vdw_ratio = 1.5
sites_z = [14]

out = input2crystal(sites_z,seed,vdw_ratio,WyckTable)
print out
print out[0].get_scaled_positions()

0.745 0.646 0.425
(Atoms(symbols='Si2', pbc=True, cell=[0.999701798109813, 7.282, 7.105]), 26, ['b'])
[[ 0.5    0.646  0.425]
 [ 0.5    0.354  0.925]]


## unskew cell

In [42]:
seed = 86
vdw_ratio = 1.5
sites_z = [14]
sg = 191
cc = input2crystal(sites_z,seed,vdw_ratio,WyckTable,sg)[0]

frame = cc.copy()
lattice = frame.get_cell()
lengths = frame.get_cell_lengths_and_angles()[:3]
angles = np.cos(frame.get_cell_lengths_and_angles()[3:]*d2r)

max_angle2ij = {0:(0,1),1:(0,2),2:(1,2)}
for max_angle in np.where(np.abs(angles)>0.5)[0]:
    
    i,j = max_angle2ij[max_angle]
    if lengths[i] > lengths[j]:
        i,j = j,i 
    lattice[j,:] = lattice[j,:] - np.round(angles[max_angle]) * lattice[i,:]
    
frame.set_cell(lattice)
frame.wrap()

print compare(cc,frame)

1.0


In [41]:
view(cc)

In [43]:
frame = unskewCell(cc)
view([frame,cc])

## LJ

In [25]:
seed = 1
vdw_ratio = 1.5
sites_z = [14]
# sg = 191
ff = 2**(1./6.)
r = z2Covalentradius[14]
sigma = 2*r/ff
r_c = 3.*sigma
print sigma,r_c

2.06688502609 6.20065507826


In [69]:
crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio)
pr = 1e-2
pressure = np.eye(3)*pr

param_str,cutoff_max = make_LJ_input(sites_z)
cc_qp = ase2qp(crystal)
pot = Potential('IP LJ',param_str=param_str)
cc_qp.set_calculator(pot)

cc_qp.get_stress()
# minimiser = Minim(cc_qp, relax_positions=True, relax_cell=True,logfile=None,method='fire',
#                   external_pressure=None,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=True)
# minimiser.run(fmax=1e-5,steps=1e6)



# view([crystal,cc_qp])


array([ -2.42259900e+04,   4.91498744e-03,   6.31512357e-03,
         4.19232592e-21,  -0.00000000e+00,  -4.19232592e-21])

In [67]:
%%time
crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio,WyckTable)
pr = 1e-0
pressure = np.eye(3)*pr

param_str = make_LJ_input(sites_z)
cc_qp = ase2qp(crystal)
pot = Potential('IP LJ',param_str=param_str)
cc_qp.set_calculator(pot)
cc_qp = LJrelax(cc_qp,FIRE,optimizerOptions={'dtmax':10.,'dt':0.1,'finc':1.1, 
                                             'fdec':0.5, 'astart':0.1, 'fa':0.99, 'a':0.1},
                runOptions={'fmax':1e-5,'steps':1e4})
cc_qp = ase2qp(cc_qp)
#view([crystal,cc_qp])

CPU times: user 15.3 s, sys: 524 ms, total: 15.9 s
Wall time: 15.9 s


In [ ]:
%%time
for seed in range(100):
    crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio,WyckTable)
    pr = 1e-0
    pressure = np.eye(3)*pr

    param_str = make_LJ_input(sites_z)
    cc_qp = ase2qp(crystal)
    pot = Potential('IP LJ',param_str=param_str)
    cc_qp.set_calculator(pot)


    minimiser = Minim(cc_qp, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                      external_pressure=None,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)
    minimiser.run(fmax=1e-5,steps=1e6)

In [16]:
Minim?

In [ ]:
dt=0.1, maxmove=0.2, dtmax=1.0, Nmin=5, finc=1.1, fdec=0.5, astart=0.1, fa=0.99, a=0.1,

In [ ]:
eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0, external_pressure=None, use_precond=0

In [32]:
crystal,sg,wki = input2crystal([14],14,1.5,WyckTable)
print sg,wki
view(crystal)

108 ['d']


In [18]:
crystal

Atoms(symbols='Si', pbc=True, cell=[[nan, nan, nan], [-1.3442709097535988, 3.9040461986495787, 0.0], [nan, nan, nan]])

In [33]:
aa = ase2qp(crystal)
print aa.get_all_distances().min()

0.0


In [34]:
crystal,sg,wki = input2crystal([14],14,1.5,WyckTable)

cc_qp = ase2qp(crystal)

Natom = cc_qp.get_number_of_atoms()
print (cc_qp.get_all_distances() + np.eye(Natom)).min()
aa.rattle(1e-10)
print (cc_qp.get_all_distances() + np.eye(Natom)).min()

0.162183963368
0.162183963193


In [33]:
seed = 5348
vdw_ratio = 1.5
sites_z = [14]

crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio,WyckTable)
pr = 1e-2
pressure = np.eye(3)*pr

param_str,max_cutoff = make_LJ_input(sites_z)
print param_str
cc_qp = ase2qp(unskewCell(crystal))
pot = Potential('IP LJ',param_str=param_str)
cc_qp.set_calculator(pot)
cc_qp.set_cutoff(max_cutoff,0.5)
Natom = cc_qp.get_number_of_atoms()

dyn = FIRE(cc_qp, logfile=None)
mf = np.linalg.norm(cc_qp.get_forces(), axis=1).max()
while mf > 1e5:
    dyn.run(**{'fmax': 3e-2, 'steps': 1})
    mf = np.linalg.norm(cc_qp.get_forces(), axis=1).max()

minimiser = Minim(cc_qp, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                  external_pressure=pressure,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

minimiser.run(fmax=1e-2,steps=1e6)

minimiser = Minim(cc_qp, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                  external_pressure=None,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

minimiser.run(fmax=1e-6,steps=1e6)

view([crystal,cc_qp])

<LJ_params n_types="1" label="default"> <per_type_data type="1" atomic_num="14" /> <per_pair_data type1="1" type2="1" sigma="2.06688502609" eps6="1.0" eps12="1.0" cutoff="6.20065507826" energy_shift="F" linear_force_shift="F" /> </LJ_params>


In [45]:
def atoms2np(crystal):
    lattice = crystal.get_cell()
    positions = crystal.get_positions()
    numbers = crystal.get_atomic_numbers()
    return numbers,positions,lattice
def np2atoms(numbers,positions,lattice,type='ase'):
    if type == 'ase':
        from ase import Atoms as aseAtoms
        crystal = aseAtoms(numbers=numbers,cell=lattice,positions=positions)
    elif type == 'quippy':
        from quippy import Atoms as qpAtoms
        crystal = qpAtoms(numbers=numbers,cell=lattice,positions=positions)
    else:
        raise TypeError('Not recognised input type {}'.format(type))
    return crystal

In [46]:
def LJ_vcrelax(sites_z,numbers,positions,lattice,isotropic_external_pressure=1e-2):
    from quippy.potential import Potential, Minim
    from ase.optimize import  FIRE
    import numpy as np
    
    crystal = np2atoms(numbers,positions,lattice,type='quippy')
    
    pressure_tensor = np.eye(3)*isotropic_external_pressure
    # get the string to setup the quippy LJ potential (parameters and species)
    param_str,max_cutoff = make_LJ_input(sites_z)
    
    pot = Potential('IP LJ',param_str=param_str)
    crystal.set_calculator(pot)
    crystal.set_cutoff(max_cutoff,0.5)
    
    # use ASE fire implementation to relax the internal d.o.g. to make sure atoms are not too close to each other 
    # when optimizing with quippy's fire implemnetation (it crashes otherwise)
    dyn = FIRE(crystal, logfile=None)
    max_force = np.linalg.norm(crystal.get_forces(), axis=1).max()
    # the threshold here make sure that quippy will not exit with out of memory error
    while max_force > 1e5:
        dyn.run(**{'fmax': 3e-2, 'steps': 1})
        max_force = np.linalg.norm(crystal.get_forces(), axis=1).max()
    
    # 1st round of vc relax with external isotropic pressure
    minimiser = Minim(crystal, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                      external_pressure=pressure_tensor,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

    minimiser.run(fmax=5e-3,steps=1e6)
    
    # 2nd round of vc relax without external isotropic pressure
    minimiser = Minim(crystal, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                      external_pressure=None,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

    minimiser.run(fmax=1e-6,steps=1e6)
    
    crystal.wrap()
    
    numbers,positions,lattice = atoms2np(crystal)
    
    return numbers,positions,lattice

## test 

In [153]:
def make_LJ_input(crystal,LJ_parameters):
    atomic_nums = np.unique(crystal.get_atomic_numbers())
    n_types = len(atomic_nums)
    types = range(1 ,n_types +1)
    ids = range(n_types)
    sigmas,epsilons,cutoffs = LJ_parameters['sigmas'],LJ_parameters['epsilons'],LJ_parameters['cutoffs']

    param_str = []
    param_str.append('<LJ_params n_types="{}" label="default">'.format(n_types))
    for tp, atomic_num in zip(types, atomic_nums):
        param_str.append('<per_type_data type="{}" atomic_num="{}" />'.format(tp, atomic_num))
    
    for it,tp1, atomic_num1 in zip(ids,types, atomic_nums):
        for jt,tp2, atomic_num2 in zip(ids,types, atomic_nums):
            if tp1 > tp2:
                continue
            ss = '<per_pair_data type1="{}" type2="{}" sigma="{}" eps6="{}" eps12="{}" cutoff="{}" energy_shift="F" linear_force_shift="F" />'.format(
                tp1, tp2, sigmas[it,jt], epsilons[it,jt], epsilons[it,jt], cutoffs[it,jt])
            param_str.append(ss)

    param_str.append('</LJ_params>')
    
    return ' '.join(param_str)

In [152]:
def get_LJ_parameters(crystal):
    # TODO: change value with formation energy and mixing rules for QMAT-2
    # https://en.wikipedia.org/wiki/Gas_constant
    # https://en.wikipedia.org/wiki/Lennard-Jones_potential
    # https://en.wikipedia.org/wiki/Combining_rules
    sites_z = np.unique(crystal.get_atomic_numbers())
    fac = 2** (1. / 6.)
    if len(sites_z) == 1:
        sigmas = np.asarray([[2 * z2Covalentradius[sites_z[0]] / fac]])
        epsilons = np.asarray([[1.0]])
        cutoffs = np.asarray([[3. * sigma]])
    
    
    return dict(sigmas=sigmas,epsilons=epsilons,cutoffs=cutoffs)

## Test the LJ vcrelax

In [147]:
def LJ_vcrelax_ase(sites_z, crystal, isotropic_external_pressure=1e-2):
    from ase.optimize import FIRE
    import numpy as np
    from quippy.potential import Potential
    #from ase.constraints import UnitCellFilter
    from libs.custom_unitcellfilter import UnitCellFilter
    # do a copy and change the object type
    crystal = ase2qp(crystal)

    #pressure_tensor = np.eye(3) * isotropic_external_pressure
    # get the string to setup the quippy LJ potential (parameters and species)
    param_str, max_cutoff = make_LJ_input(sites_z)

    pot = Potential('IP LJ', param_str=param_str)
    crystal.set_calculator(pot)
    crystal.set_cutoff(max_cutoff, 0.5)

    # use ASE fire implementation to relax the internal d.o.g. to make sure atoms are not too close to each other
    # when optimizing with quippy's fire implemnetation (it crashes otherwise)
    # dyn = FIRE(crystal, logfile=None)
    # max_force = np.linalg.norm(crystal.get_forces(), axis=1).max()

    V = crystal.get_volume()
    N = crystal.get_number_of_atoms()
    J = V ** (1 / 3.) * N ** (1 / 6.)
    ucf = UnitCellFilter(crystal, mask=[1, 1, 1, 1, 1, 1], cell_factor=V / J, hydrostatic_strain=False,
                         constant_volume=False,isotropic_pressure=isotropic_external_pressure)
    dyn = FIRE(ucf, logfile=None)
    # max_force = np.linalg.norm(crystal.get_forces(), axis=1).max()

    dyn.run(**{'fmax': 1e-6, 'steps': 1e5})
    
    crystal.wrap()
    
    return crystal

In [148]:
def LJ_vcrelax_qp(sites_z,crystal,isotropic_external_pressure=1e-2):
    from quippy.potential import Potential, Minim
    from ase.optimize import  FIRE
    import numpy as np
    
    crystal = ase2qp(crystal)
    
    if isotropic_external_pressure == 0.:
        pressure_tensor = None
    else:
        pressure_tensor = np.eye(3)*isotropic_external_pressure
    # get the string to setup the quippy LJ potential (parameters and species)
    param_str,max_cutoff = make_LJ_input(sites_z)
    
    pot = Potential('IP LJ',param_str=param_str)
    crystal.set_calculator(pot)
    crystal.set_cutoff(max_cutoff,0.5)
    
    
    # 2nd round of vc relax without external isotropic pressure
    minimiser = Minim(crystal, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                      external_pressure=pressure_tensor,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

    minimiser.run(fmax=1e-6,steps=1e6)
    
    crystal.wrap()
    
    return crystal

In [150]:
seed = 2
vdw_ratio = 1.5
sites_z = [14]

crystal, sg, wki = input2crystal(sites_z, seed, vdw_ratio)
crystal = unskewCell(crystal)

p  = 0.

cc_qp = LJ_vcrelax_qp(sites_z,crystal,isotropic_external_pressure=p)

cc_ase = LJ_vcrelax_ase(sites_z, crystal, isotropic_external_pressure=p)

print np.allclose(cc_qp.get_scaled_positions(),cc_ase.get_scaled_positions(),atol=1e-5)
print np.allclose(cc_qp.get_cell(),cc_ase.get_cell(),atol=1e-5)

True
True


In [143]:
view([crystal,cc_ase,cc_qp])

## test generation results

In [31]:
from generate_and_relax_structures import generate_crystal_step_1,input2crystal
import spglib as spg
from tqdm import tqdm_notebook

In [50]:
sg = 214
n = 200000
inputs = [dict(sites_z=[14], seed=seed, vdw_ratio=1.5, 
               isotropic_external_pressure=1e-3, symprec=1e-05,sg =sg) for seed in range(n,n+50)]
frames = []
for inp in tqdm_notebook(inputs):
    cc = generate_crystal_step_1(**inp)
    if cc is not None:
        frames.append(cc)
        sym_data = spg.get_symmetry_dataset(cc)
        print '{}\t{}\t{}'.format(spg.get_spacegroup(cc),len(np.unique(sym_data['equivalent_atoms'])),
                                  np.unique(sym_data['wyckoffs']))


HBox(children=(IntProgress(value=0, max=50), HTML(value=u'')))

I4_132 (214)	1	['h']
Im-3m (229)	1	['a']
Im-3m (229)	1	['a']
I4_132 (214)	1	['g']
I4_132 (214)	1	['b']
I4_132 (214)	1	['a']
I4_132 (214)	1	['d']
I4_132 (214)	1	['d']
Im-3m (229)	1	['a']
I4_132 (214)	1	['g']
Im-3m (229)	1	['a']
I4_132 (214)	1	['f']
I4_132 (214)	1	['d']
I4_132 (214)	1	['c']
I4_132 (214)	1	['a']
I4_132 (214)	1	['b']
I4_132 (214)	1	['c']
I4_132 (214)	1	['a']
I4_132 (214)	1	['g']
I4_132 (214)	1	['g']
I4_132 (214)	1	['a']
I4_132 (214)	1	['f']
I4_132 (214)	1	['d']
I4_132 (214)	1	['b']
I4_132 (214)	1	['g']
I4_132 (214)	1	['c']
I4_132 (214)	1	['d']
I4_132 (214)	1	['c']
Im-3m (229)	1	['a']
I4_132 (214)	1	['i']
I4_132 (214)	1	['g']
I4_132 (214)	1	['h']
I4_132 (214)	1	['g']
I4_132 (214)	1	['g']
I4_132 (214)	1	['g']
I4_132 (214)	1	['d']
I4_132 (214)	1	['b']
Im-3m (229)	1	['a']
I4_132 (214)	1	['b']
I4_132 (214)	1	['d']
I4_132 (214)	1	['i']
I4_132 (214)	1	['d']
I4_132 (214)	1	['i']
I4_132 (214)	1	['g']
I4_132 (214)	1	['i']
I4_132 (214)	1	['a']
I4_132 (214)	1	['i']
I4_132 (214)	1	['c'

In [63]:
print sym_data
print type(sym_data['number'])

{'std_positions': array([[ 0.98095084,  0.375     ,  0.23095084],
       [ 0.98095084,  0.625     ,  0.26904916],
       [ 0.01904916,  0.125     ,  0.23095084],
       [ 0.51904916,  0.375     ,  0.76904916],
       [ 0.26904916,  0.98095084,  0.625     ],
       [ 0.23095084,  0.98095084,  0.375     ],
       [ 0.76904916,  0.51904916,  0.375     ],
       [ 0.23095084,  0.01904916,  0.125     ],
       [ 0.375     ,  0.76904916,  0.51904916],
       [ 0.125     ,  0.23095084,  0.01904916],
       [ 0.625     ,  0.26904916,  0.98095084],
       [ 0.375     ,  0.23095084,  0.98095084],
       [ 0.48095084,  0.875     ,  0.73095084],
       [ 0.48095084,  0.125     ,  0.76904916],
       [ 0.51904916,  0.625     ,  0.73095084],
       [ 0.01904916,  0.875     ,  0.26904916],
       [ 0.76904916,  0.48095084,  0.125     ],
       [ 0.73095084,  0.48095084,  0.875     ],
       [ 0.26904916,  0.01904916,  0.875     ],
       [ 0.73095084,  0.51904916,  0.625     ],
       [ 0.875     ,  

In [65]:
np.unique?

In [46]:
view(frames)

In [79]:
print range(3,231)

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223

## save frames in h5

In [177]:
with open('./test.pck','rb') as f:
    frames = pck.load(f)

In [176]:
import h5py

In [53]:
f = h5py.File("mytestfile.hdf5", "w")

IOError: Unable to create file (Unable to truncate a file which is already open)

In [43]:
grp = f.create_group("subgroup")

In [45]:
frame = frames[0]

In [74]:
lattice = frame.get_cell()
pos = frame.get_positions()
num = frame.get_atomic_numbers()
perio = np.array([True,]*3)
print lattice,pos,num,perio

[[ 2.25991981  0.          0.        ]
 [-1.1299599   1.95714796  0.        ]
 [ 0.          0.          3.69072859]] [[ 1.1299599   0.65238265  0.92268215]
 [ 0.          1.30476531  2.76804644]] [14 14] [ True  True  True]


In [95]:
with h5py.File("mytestfile.hdf5", "a",libver='latest') as f:
    f.swmr_mode = True
    grp = f.create_group("frame_3")
    cell = grp.create_dataset("cell", data=lattice)
    positions = grp.create_dataset("positions",data=pos)
    numbers = grp.create_dataset("numbers",data=num)
    pbc = grp.create_dataset("pbc",data=perio)

In [71]:
data = {}
with h5py.File("mytestfile.hdf5", "r",libver='latest',swmr=True) as f:
    for fr_name,grp in f.iteritems():
        print fr_name
        data[fr_name] = {}
        for name,dset in grp.iteritems():
            
            data[fr_name][name] = dset.value
            print data[fr_name][name]
frame = ase.Atoms(**data['frame_1'])
print frame

frame_1
[[ 2.25991981  0.          0.        ]
 [-1.1299599   1.95714796  0.        ]
 [ 0.          0.          3.69072859]]
[[ 1.1299599   0.65238265  0.92268215]
 [ 0.          1.30476531  2.76804644]]
[14 14]
[False False False]
Atoms(symbols='Si2', pbc=False, cell=[[2.2599198074019387, 0.0, 0.0], [-1.1299599037009693, 1.9571479637257145, 0.0], [0.0, 0.0, 3.6907285872261926]])


In [104]:
with h5py.File("mytestfile.hdf5", "r",libver='latest') as f:
    print f.keys()

[u'frame_1', u'frame_2', u'frame_3']


In [178]:
a = [234,234,23423,234445,55]
print a[:-1]

[autoreload of generate_and_relax_structures failed: Traceback (most recent call last):
  File "/home/musil/miniconda/envs/glosim/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "generate_and_relax_structures.py", line 37
    for st,nd in zip(strides[:-1])
                                 ^
SyntaxError: invalid syntax
]


[234, 234, 23423, 234445]


In [117]:
class Frame_Dataset_h5(object):
    def __init__(fname,swmr_mode=True,bname="frame"):
        self.fname = fname
        
        self.f = h5py.File(fname, 'w',libver='latest')
        self.f.swmr_mode = swmr_mode
        self.f.close()
        
        self.swmr_mode = swmr_mode
        self.counter = 0
        self.frame_fields = ["cell","positions","numbers","pbc"]
        
        self.bname = bname
        
    def dump_frame(self,f,crystal,inp_dict):
        grp = f.create_group(self.bname+'_{}'.format(self.counter))
        cell = grp.create_dataset("cell", data=crystal.get_cell())
        positions = grp.create_dataset("positions",data=crystal.get_positions())
        numbers = grp.create_dataset("numbers",data=crystal.get_atomic_numbers())
        pbc = grp.create_dataset("pbc",data=crystal.get_pbc())
        
        sgrp = grp.create_group("inputs")
        for name,val in inp_dict.iteritems():
            sgrp.create_dataset(name, data=np.array(val))
        self.counter += 1
        
    def dump_frames(self,crystals,inputs):
        with h5py.File(self.fname, 'a',libver='latest') as f:
            for crystal,inp_dict in zip(crystals,inputs):
                self.dump_frame(f,crystal,inp_dict)
                
    def load_frame(self,fname,name,frame_type='quippy'):
        data = {}
        
        with h5py.File(self.fname, "r",libver='latest', swmr=self.swrm_mode) as f:
            for field in self.frame_fields:
                data[dname] = f[name][field].value
        if frame_type == 'quippy':
            from quippy import Atoms as qpAtoms
            frame = qpAtoms(**data)
        elif frame_type == 'ase':
            from ase import Atoms as aseAtoms
            frame = aseAtoms(**data)
        return frame
    
    def load_frames(self,fname,name,frame_type='quippy'):
        data = {}
        with h5py.File(self.fname, "r",libver='latest', swmr=self.swrm_mode) as f:
            for name in names:
                for field in self.frame_fields:
                    data[name][dname] = f[name][field].value
        frames = {}
        if frame_type == 'quippy':
            from quippy import Atoms as qpAtoms
            for name in names:
                frames[name] = qpAtoms(**data[name])
        elif frame_type == 'ase':
            from ase import Atoms as aseAtoms
            for name in names:
                frames[name] = aseAtoms(**data[name])
        
        return frames
    
        

In [182]:
with open('./test.pck','rb') as f:
    frames = pck.load(f)

In [183]:
from libs.io import Frame_Dataset_h5

In [235]:
fname = './relaxed_structures_step1-0.h5'
fout = Frame_Dataset_h5(fname)
print fout.get_names()

[]


In [227]:
# fout = Frame_Dataset_h5(fname)
fff = fout.load_frames()
# view(fff)

In [228]:
fff

{u'frame_0': <Atoms object at 0x7f3409350250 fpointer=(134158944,)>,
 u'frame_1': <Atoms object at 0x7f3409301290 fpointer=(138160992,)>,
 u'frame_10': <Atoms object at 0x7f34092ef850 fpointer=(134964976,)>,
 u'frame_11': <Atoms object at 0x7f3409313110 fpointer=(135146704,)>,
 u'frame_12': <Atoms object at 0x7f340933e310 fpointer=(135789376,)>,
 u'frame_13': <Atoms object at 0x7f340933e610 fpointer=(134372848,)>,
 u'frame_14': <Atoms object at 0x7f3409301190 fpointer=(135141968,)>,
 u'frame_15': <Atoms object at 0x7f340933e110 fpointer=(136144016,)>,
 u'frame_16': <Atoms object at 0x7f3409333c50 fpointer=(135611664,)>,
 u'frame_17': <Atoms object at 0x7f3409301a90 fpointer=(87755536,)>,
 u'frame_18': <Atoms object at 0x7f34092a3610 fpointer=(135100816,)>,
 u'frame_19': <Atoms object at 0x7f34093506d0 fpointer=(137955200,)>,
 u'frame_2': <Atoms object at 0x7f34092b4110 fpointer=(134707680,)>,
 u'frame_20': <Atoms object at 0x7f3409301610 fpointer=(135136624,)>,
 u'frame_21': <Atoms obj

In [92]:
h5py.version.info

'Summary of the h5py configuration\n---------------------------------\n\nh5py    2.7.0\nHDF5    1.10.1\nPython  2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:09:15) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]\nsys.platform    linux2\nsys.maxsize     9223372036854775807\nnumpy   1.13.3\n'

In [234]:
import os
''.join(os.path.abspath(fname).split('.')[:-1])

'/local/git/generate_structures/test'

In [115]:
get_frame_h5("mytestfile.hdf5",'frame_1')

<Atoms object at 0x7f3409903550 fpointer=(75197440,)>

## glosim

In [38]:
centerweight  = 1.
gaussian_width = 0.5
cutoff = 3.5
cutoff_transition_width = 0.5
nmax = 20
lmax = 15
nocenters = []
is_fast_average = True

soap_params = {
          'centerweight': centerweight, 
          'gaussian_width': gaussian_width,'cutoff': cutoff, 
          'cutoff_transition_width': cutoff_transition_width,
          'nmax': nmax, 'lmax': lmax, 'is_fast_average':is_fast_average,
          'chem_channels': True ,'nocenters': nocenters,'dispbar':False,
               }


In [ ]:
dataPath = '/local/gdrive/Musil/qmat/structures/step1/'
fns = [dataPath + 'relaxed_structures_step1_9-0.h5',
      dataPath + 'relaxed_structures_step1_9-1.h5']

In [31]:
view(frames)

In [39]:
fings = get_Soaps(frames,nprocess=4, **soap_params)

HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=900), HTML(value=u'')))

# general scheme

In [173]:
seed = 54565
vdw_ratio = 1.5
sites_z = [14]

In [174]:
crystal, sg, wki = input2crystal(sites_z, seed, vdw_ratio)
view(crystal)
crystal = unskewCell(crystal)

crystal = LJ_vcrelax(crystal,isotropic_external_pressure=1e-3,debug=False)
view(crystal)
crystal = get_standard_frame(crystal, to_primitive=False, symprec=1e-5)
view(crystal)

In [175]:
data = spg.get_symmetry_dataset(crystal,symprec=1e-5)
print data.keys()
print sg,wki
print data['number'],data['wyckoffs'],data['equivalent_atoms']

['std_positions', 'equivalent_atoms', 'std_types', 'translations', 'rotations', 'number', 'choice', 'transformation_matrix', 'std_mapping_to_primitive', 'origin_shift', 'hall_number', 'mapping_to_primitive', 'pointgroup', 'international', 'wyckoffs', 'hall', 'std_lattice']
34 ['c']
139 ['b', 'b'] [0 0]


In [ ]:
%%time
vdw_ratio = 1.5
sites_z = [14]

inputs = [[sites_z,seed,vdw_ratio] for seed in range(100)]
res = []
for inp in tqdm_notebook(inputs):
    res.append(generate_crystal_step_1_wrapper(inp))


HBox(children=(IntProgress(value=0), HTML(value=u'')))

0.775 0.786 0.778


In [ ]:
view(res[100])

## test special cases

In [18]:
sites_z,seed,vdw_ratio = [[14], 1, 1.5]
crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio)
view(crystal)
crystal = unskewCell(crystal)
view(crystal)

0.676 0.1 0.342


In [8]:
print crystal.get_cell_lengths_and_angles()

[   0.55899479    9.78          6.03389907   98.83400165  125.93040169
   94.        ]


In [9]:
sg,wki

(1, ['a'])

In [22]:
crystal = unskewCell(crystal)

crystal = LJ_vcrelax(sites_z,crystal,isotropic_external_pressure=1e-2)

## make atom separator

### make the function

In [8]:
seed = 101
vdw_ratio = 1.5
sites_z = [14]

In [9]:
from libs.raw_data import z2Covalentradius,separationData
from scipy.spatial.distance import pdist,cdist,squareform
from ase.neighborlist import NeighborList

In [10]:
crystal, sg, wki = input2crystal(sites_z, seed, vdw_ratio,sg=227)
crystal = unskewCell(crystal)
view(crystal)

In [147]:
cov_rad = []
for z in sites_z:
    cov_rad.append(z2Covalentradius[z])

In [210]:
sc_pos = crystal.get_scaled_positions()
numbers = crystal.get_atomic_numbers()
distance_lin = pdist(sc_pos, metric='euclidean')
distance = squareform(distance_lin)

In [211]:
distance[distance<1e-6]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [161]:
NeighborList?

In [178]:
nl = NeighborList([z2Covalentradius[z] for z in numbers],self_interaction=True, bothways=False)
nl.build(crystal)



True

In [181]:
positions = []
indices, offsets = nl.get_neighbors(42)
for i, offset in zip(indices, offsets):
    crystal.positions[i] + dot(offset, crystal.get_cell())

In [196]:
aa = np.random.rand(5,3)
print aa
aa /= np.linalg.norm(aa,axis=1).reshape((-1,1))
print aa

[[ 0.91449735  0.53982848  0.33657434]
 [ 0.98218914  0.0779967   0.10080531]
 [ 0.55138367  0.45981604  0.54715623]
 [ 0.71436115  0.03939172  0.9195967 ]
 [ 0.00749051  0.50132383  0.14011776]]
[[ 0.8209111   0.48458444  0.30213057]
 [ 0.99168501  0.07875078  0.1017799 ]
 [ 0.61082829  0.50938877  0.6061451 ]
 [ 0.61311854  0.03380894  0.78926714]
 [ 0.01438848  0.96299037  0.26915149]]


In [12]:
atoms = crystal.copy()
Natom = atoms.get_number_of_atoms()
numbers = atoms.get_atomic_numbers()
dr = np.zeros((Natom,3))
r = atoms.get_positions()
cell = atoms.get_cell()

nl = NeighborList([z2Covalentradius[z] for z in numbers],skin=0.,self_interaction=False, bothways=False)
nl.build(atoms)

for icenter in range(Natom):
    indices, offsets = nl.get_neighbors(icenter)
    
    icenter_pos = r[icenter].reshape((1,3))
    neighbor_pos = np.zeros((len(indices),3))
#     print indices
    for it,(ineighbor, offset) in enumerate(zip(indices, offsets)):
        neighbor_pos[it] = r[ineighbor] + np.dot(offset, cell)
    
    sepVec = icenter_pos - neighbor_pos
    sep = np.linalg.norm(sepVec,axis=1).reshape((-1,1))
    sepDiff = separationData[(numbers[icenter],numbers[ineighbor])] - sep
    
    for it,ineighbor in enumerate(indices):
#         print it,sepDiff[it],sep[it],sepVec[it]
#         print icenter,dr[icenter]
        dr[icenter] += 0.5 * sepDiff[it] / sep[it] * sepVec[it]
        dr[ineighbor] -= 0.5 * sepDiff[it] / sep[it] * sepVec[it]
    
atoms.set_positions(r+dr)
view(crystal)
atoms.wrap()
view(atoms)

In [266]:
class AtomSeparator(object):
    def __init__(self,atoms,tol=1e-6):
        super(AtomSeparator,self).__init__()
        self.atoms = ase2qp(atoms)
        self.numbers = atoms.get_atomic_numbers()
        self.Natom = atoms.get_number_of_atoms()
        self.nl = NeighborList([z2Covalentradius[z] for z in numbers],self_interaction=False, bothways=False)
        self.has_neighbors = np.ones((Natom),dtype=bool)
        self.tol = tol
    
    def run(self,Nmax = 50):
        ii = 0
        fff = True
        while fff:
            fff = self.step()
            ii += 1
            if ii >= Nmax:
                fff = False
    
    def step(self):
        atoms = self.atoms
        nl = self.nl
        
        nl.update(atoms)
                
        Natom = self.Natom
        numbers = self.numbers
        dr = np.zeros((Natom,3))
        r = atoms.get_positions()
        cell = atoms.get_cell()
        
        for icenter in range(Natom):
            
            indices, offsets = nl.get_neighbors(icenter)
            icenter_pos = r[icenter].reshape((1,3))
            neighbor_pos = np.zeros((len(indices),3))
            refSep = np.zeros((len(indices),1))
            for it,(ineighbor, offset) in enumerate(zip(indices, offsets)):
                neighbor_pos[it] = r[ineighbor] + np.dot(offset, cell)
                refSep[it] = separationData[(numbers[icenter], numbers[ineighbor])]

            sepVec = np.subtract( icenter_pos, neighbor_pos)
            sep = np.linalg.norm(sepVec,axis=1).reshape((-1,1))
            sepDiff = np.subtract(refSep, sep)
        
            move = np.multiply(0.5, np.multiply(np.divide(sepDiff, sep), sepVec))
            
            for it,ineighbor in enumerate(indices):
                dr[icenter] += move[it]
                dr[ineighbor] -= move[it]
        
        if np.linalg.norm(dr) < self.tol:
            return False
        else:
            atoms.set_positions(r+dr)
            atoms.wrap()
            return True

In [267]:
atoms = crystal.copy()
opt = AtomSeparator(atoms)

opt.opt()
view(crystal)
view(atoms)


In [240]:
view(atoms)

In [229]:
from libs.LJ_pressure import LJ_vcrelax_ase_simple

In [230]:
cc = crystal.copy()
cc = LJ_vcrelax_ase_simple(cc,isotropic_external_pressure=None)
view(cc)

In [223]:
cc

Atoms(symbols='Si96', pbc=True, cell=[13.540732413738343, 13.540732413738343, 13.540732413738343])

In [236]:
bb = np.array([], dtype=np.int64)
nn = nl.neighbors
for it,n in enumerate(nn):
    if n.size == 0:
        print it
print nn

63
72
76
80
84
85
86
87
88
89
90
91
92
93
94
95
[array([36, 57, 66, 78, 93]), array([22, 40, 58, 61, 76]), array([17, 44, 56, 77, 86]), array([15, 59, 71, 79, 91]), array([34, 37, 64, 73, 94]), array([14, 32, 68, 74, 89]), array([21, 33, 42, 60, 72]), array([19, 35, 47, 75, 87]), array([29, 38, 50, 65, 92]), array([18, 30, 45, 48, 84]), array([13, 28, 49, 70, 88]), array([23, 31, 43, 51, 63]), array([24, 54, 69, 81, 90]), array([28, 49, 70, 88]), array([32, 68, 74, 89]), array([59, 71, 79, 91]), array([25, 46, 52, 82, 85]), array([44, 56, 77, 86]), array([30, 45, 48, 84]), array([35, 47, 75, 87]), array([26, 41, 53, 62, 80]), array([33, 42, 60, 72]), array([40, 58, 61, 76]), array([31, 43, 51, 63]), array([54, 69, 81, 90]), array([46, 52, 82, 85]), array([41, 53, 62, 80]), array([39, 55, 67, 83, 95]), array([49, 70, 88]), array([38, 50, 65, 92]), array([45, 48, 84]), array([43, 51, 63]), array([68, 74, 89]), array([42, 60, 72]), array([37, 64, 73, 94]), array([47, 75, 87]), array([57, 

### benchmark it

In [9]:
from libs.LJ_pressure import LJ_vcrelax_alternative,LJ_vcrelax,AtomSeparator

In [7]:
%load_ext line_profiler

In [21]:
seed = 110
vdw_ratio = 1.5
sites_z = [14]

In [22]:
%%time
crystal, sg, wki = input2crystal(sites_z, seed, vdw_ratio,sg=227)
crystal = unskewCell(crystal)
cc = LJ_vcrelax(crystal,isotropic_external_pressure=1e-2,debug=True)
view(cc)

5678
5890
CPU times: user 3min 19s, sys: 1.51 s, total: 3min 20s
Wall time: 3min 19s


In [23]:
%%time
from libs.LJ_pressure import LJ_vcrelax_alternative,LJ_vcrelax,AtomSeparator
seed = 110
vdw_ratio = 1.5
sites_z = [14]
crystal, sg, wki = input2crystal(sites_z, seed, vdw_ratio,sg=227)
crystal = unskewCell(crystal)
sep = AtomSeparator(crystal)
sep.run()
view(sep.atoms)
cc = LJ_vcrelax_alternative(sep.atoms,isotropic_external_pressure=1e-2,debug=True)
view(cc)

983
1002
CPU times: user 52.4 s, sys: 520 ms, total: 52.9 s
Wall time: 40.2 s


In [29]:
data = spg.get_symmetry_dataset(cc)
print data['number'],data['wyckoffs']

1 ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']


In [285]:
crystal, sg, wki = input2crystal(sites_z, seed, vdw_ratio,sg=227)
crystal = unskewCell(crystal)
sep = AtomSeparator(crystal)
sep.run()

%lprun -f LJ_vcrelax_alternative LJ_vcrelax_alternative(sep.atoms,isotropic_external_pressure=1e-2,debug=True)

243
217


In [17]:
import spglib as spg

from libs.LJ_pressure import LJ_vcrelax_alternative,LJ_vcrelax,AtomSeparator
seed = 110
vdw_ratio = 1.
sites_z = [14]
crystal, sg, wki = input2crystal(sites_z, seed, vdw_ratio,sg=227)
data = spg.get_symmetry_dataset(crystal)
print data['number'],data['wyckoffs']
crystal = unskewCell(crystal)
sep = AtomSeparator(crystal)
sep.run()

data = spg.get_symmetry_dataset(sep.atoms)
print data['number'],data['wyckoffs']

227 ['i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i']
1 ['a', 'a', 'a', 'a', 'a', 'a', 'a

In [18]:
view(crystal)